In [1]:
import sys
import random
import pickle

import pandas as pd
import numpy as np
import json
from konlpy.tag import Twitter
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB , GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
# read train data
with open('./data/train_data.json') as fp:
    json_str = fp.read()
    json_data = json.loads(json_str)
    
# convert to dataframe
train_df = pd.DataFrame(json_data)

# train data preprocessing
train_df['rating_cat'] = train_df['rating'].apply(lambda x: 
                    'NEG' if 1<= x <=3 
                     else 
                      ('NEU' if 4<=x<=7 
                     else 'POS'))

# read test data in/out
with open('./data/test.input') as fp:
    test_in = fp.read()
    test_in = test_in.splitlines()

with open('./data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()


In [4]:
# vectorize train

twitter = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), use_idf=False, smooth_idf=False)


y = train_df.rating_cat
X = vectorizer.fit_transform(train_df.review)

In [ ]:
print('a')

In [19]:
# loss function 변경
#model = SGDClassifier(alpha=1.9e-6, n_iter=19).fit(X, y)
model = LogisticRegression(C=30).fit(X,y)

In [28]:
model1 = SGDClassifier(loss='modified_huber',alpha=1.9e-6, n_iter=19)
model2 = SGDClassifier(loss='modified_huber',alpha=3.9e-6, n_iter=19)
model3 = SGDClassifier(loss='modified_huber',alpha=2.9e-6, n_iter=19)
model4 = SGDClassifier(loss='modified_huber',alpha=2.9e-6, n_iter=19)
model5 = SGDClassifier(loss='modified_huber',alpha=2.9e-6, n_iter=19)
model6 = SGDClassifier(loss='modified_huber',alpha=2.9e-6, n_iter=19)

#model3 = LinearSVC(C=0.5)#LinearSVC(C=0.5)
ensemble = VotingClassifier(estimators=[('svc1', model1), ('svc2', model2), ('svc3', model3)
                                    , ('svc4', model4)  , ('svc5', model5), ('svc6', model6) ], 
                           voting='soft')


In [29]:
model = ensemble.fit(X,y)

In [6]:
feature_list = vectorizer.get_feature_names()

test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), vocabulary = feature_list)
X_test = test_vectorizer.fit_transform(test_in[:8400])


test_pred = model.predict(X_test)

In [14]:
test_pred = model.predict(X_test)

In [49]:
#model = ensemble.fit(X, y)
model = SGDClassifier(alpha=2.9e-6, n_iter=19).fit(X, y)

In [15]:
model = ensemble.fit(X,y)

In [35]:
model = LinearSVC(C=0.2).fit(X, y)

In [20]:
# vectorize test_x, predict test_y
with open('./data/test.input') as fp:
    test_in = fp.read()
    test_in = test_in.splitlines()
    
feature_list = vectorizer.get_feature_names()

test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), vocabulary = feature_list)
X_test = test_vectorizer.fit_transform(test_in[:8400])

test_pred = model.predict(X_test)

In [25]:
# submission
with open('./data/grading.input') as fp:
    sbm_in = fp.read()
    sbm_in = sbm_in.splitlines()
 
feature_list = vectorizer.get_feature_names()

test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), vocabulary = feature_list)
sbm_test = test_vectorizer.fit_transform(sbm_in[:8400])

sbm_pred = model.predict(sbm_test)

output_file = 'submission.txt'#sys.argv[2] # 


with open(output_file, 'w') as fp:
    for _test_pred in sbm_pred:
        fp.write(_test_pred)
        fp.write('\n')
        

In [30]:
sbm_pred = model.predict(sbm_test)
output_file = 'submission.txt'#sys.argv[2] # 


with open(output_file, 'w') as fp:
    for _test_pred in sbm_pred:
        fp.write(_test_pred)
        fp.write('\n')
        

In [21]:
# logistic c30
with open('./data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.58      0.55      0.56      1022
        NEU       0.51      0.46      0.48      1825
        POS       0.84      0.87      0.86      5553

avg / total       0.74      0.74      0.74      8400

0.744047619048
0.743
0.76
0.715
0.755833333333


In [12]:
# linear sgd 1.9 5
with open('./data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.62      0.57      0.60      1022
        NEU       0.65      0.40      0.49      1825
        POS       0.82      0.94      0.88      5553

avg / total       0.76      0.78      0.76      8400

0.775833333333
0.7745
0.7925
0.746
0.787916666667


In [9]:
# linear sgd 3.9 e
with open('./data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.65      0.52      0.58      1022
        NEU       0.69      0.34      0.46      1825
        POS       0.80      0.96      0.87      5553

avg / total       0.76      0.77      0.75      8400

0.772142857143
0.7815
0.78
0.736
0.787916666667


In [6]:
# linear svc 1e6 19
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.62      0.57      0.59      1022
        NEU       0.62      0.41      0.49      1825
        POS       0.83      0.93      0.88      5553

avg / total       0.76      0.77      0.76      8400

0.77369047619
0.7765
0.781
0.745
0.789166666667


In [40]:
# linear svc 0.2 
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.65      0.54      0.59      1022
        NEU       0.61      0.42      0.50      1825
        POS       0.82      0.93      0.87      5553

avg / total       0.76      0.77      0.76      8400

0.773214285714
0.78
0.78
0.743
0.787083333333


In [26]:
# test out sgd (3.9*e-6) #high score # 최대
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.64      0.54      0.59      1022
        NEU       0.66      0.38      0.48      1825
        POS       0.81      0.95      0.87      5553

avg / total       0.76      0.77      0.75      8400

0.774285714286
0.782
0.7825
0.741
0.78875


In [6]:
# test out sgd (3.9*e-6) #high score # 최대
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.63      0.56      0.59      1022
        NEU       0.67      0.39      0.49      1825
        POS       0.82      0.94      0.88      5553

avg / total       0.76      0.78      0.76      8400

0.777023809524
0.783
0.785
0.7505
0.7875


In [38]:
with open('/elice/data/test.input') as fp:
    test_in = fp.read()
    test_in = test_in.splitlines()

In [82]:
#model = ensemble.fit(X, y)
model = SGDClassifier(alpha=0.0000005, n_iter=19).fit(X, y)

In [86]:
# submission
with open('/elice/data/grading.input') as fp:
    test_in = fp.read()
    test_in = test_in.splitlines()
 
feature_list = vectorizer.get_feature_names()

test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), vocabulary = feature_list)
X_test = test_vectorizer.fit_transform(test_in[:8400])

test_pred = model.predict(X_test)

In [87]:
output_file = 'submission.txt'#sys.argv[2] # 


with open(output_file, 'w') as fp:
    for _test_pred in test_pred:
        fp.write(_test_pred)
        fp.write('\n')

In [ ]:
# train, cross validation

n = np.arange(15,25)


ite = 5 #  반복
score = np.zeros([len(n), ite])


for i, val in enumerate(n):
    for k in range(ite): 
        
        model = SGDClassifier(alpha=2.9e-6, n_iter=val).fit(X, y)
        test_pred = model.predict(X_test)

        score[i][k] = accuracy_score(test_out[:8400], test_pred)
        
print(score)
print(score.mean(axis=1))

In [7]:
#alpha 최적값 찾기
alpha = np.arange(0.0000039,0.000004, 0.00000001)
alpha = np.arange(0.000001,0.00001,0.000001) ## 3e-6 이 일등
alpha = np.arange(2.5e-6,3.5e-6,1e-7) ## 2.9e-6이 일등

alpha = [0.000001,0.000002, 0.000003]


ite = 3 #  반복
score = np.zeros([len(alpha), ite])

for i, val in enumerate(alpha):
    for k in range(ite): 
        model = SGDClassifier(alpha=val).fit(X, y)
        test_pred = model.predict(X_test)

        score[i][k] = accuracy_score(test_out[:8400], test_pred)
        
print(score)
print(score.mean(axis=1))

[[ 0.77119048  0.77095238  0.76952381]
 [ 0.77440476  0.77333333  0.77571429]
 [ 0.775       0.77297619  0.77583333]]
[ 0.77055556  0.77448413  0.77460317]


In [ ]:
score.max()

In [ ]:
alpha = np.arange(2.5e-6,3.5e-6,1e-7)
alpha

In [ ]:
score

In [ ]:
a=  np.array([[1,1],[2,2],[3,3]])
a
a.mean(axis=1)

In [ ]:
score

In [ ]:
score.mean(axis=1)

In [ ]:
alpha = np.arange(0.0000039,0.000004, 0.00000001)

for i in alpha:
    model = SGDClassifier(alpha=i).fit(X, y)
    test_pred = model.predict(X_test)

    print(accuracy_score(test_out[:8400], test_pred))

In [ ]:
alpha = np.arange(0.0000039,0.000004, 0.00000001)
a = (0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001)
for i in alpha:
    model = SGDClassifier(alpha=i).fit(X, y)
    test_pred = model.predict(X_test)

    print(accuracy_score(test_out[:8400], test_pred))

In [ ]:
alpha = (0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001)
for i in alpha:
    model = SGDClassifier(alpha=i).fit(X, y)
    test_pred = model.predict(X_test)

    print(accuracy_score(test_out[:8400], test_pred))

In [ ]:
model = SGDClassifier(alpha=0.0000039).fit(X, y)

In [ ]:
test_pred = model.predict(X_test)

In [ ]:
# test out sgd (3.9*e-6) #high score # 최대
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (3.9*e-6) #high score # 최대
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (3.5*e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (4*e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (3*e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (7*e-5)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (5*e-5)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (1.e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (1.e-5)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd (1.e-4)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

In [ ]:
# test out sgd 
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()


#t(koken twit pos) - s
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


In [ ]:
# test out sgd 
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()


#t(koken twit pos) - s
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


In [ ]:
# 실패
from konlpy.tag import Twitter
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

                

model = Pipeline([
            ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2))), 
            ('mb',   SVC(kernel='linear')),
        ])


X = train_df.review
y = train_df.rating_cat

model.fit(X, y)

In [ ]:
# 최고기록 0.7730
from konlpy.tag import Twitter
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

                
model = Pipeline([
            ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2))), 
            ('mb',   LogisticRegression()),
        ])


X = train_df.review
y = train_df.rating_cat

model.fit(X, y)

In [ ]:
# 낮음
model = Pipeline([
            ('vect', TfidfVectorizer()), 
            ('mb', LogisticRegression(C=10.0)),
        ])

X = train_df.review
y = train_df.rating_cat

model.fit(X, y)

In [ ]:
train_df['review_nouns'][:10]

In [ ]:
train_df['review_nouns'][:10].astype(str).apply(lambda x: ' '.join(x))

In [ ]:
from konlpy.tag import *
twitter = Twitter()

In [ ]:
train_df['review_nouns'] = train_df['review'].apply(twitter.nouns)

In [ ]:
train_df.to_csv('train_df.csv')

In [ ]:
twitter.nouns('안녕하세요 저는 사람입니다. 사랑합니다.')

In [ ]:
#input_file = '/elice/data/grading.input'#sys.argv[1] #'/elice/data/test.input' #
input_file = '/elice/data/test.input' 
output_file = 'submission.txt'#sys.argv[2] # 




with open(input_file) as fp:
    test_in = fp.read()

test_in = test_in.splitlines()
test_pred = model.predict(test_in[:8400])


with open(output_file, 'w') as fp:
    for _test_pred in test_pred:
        fp.write(_test_pred)
        fp.write('\n')
        
#t(koken twit pos) - s
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


In [ ]:
from konlpy.tag import Twitter
pos_tagger = Twitter()

In [ ]:
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

In [ ]:
#t-l(koken twit pos)
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


In [ ]:
#t-l(c=10))
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))



In [ ]:
#t-linear svc
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))

In [ ]:
#t-linear svc (5)
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))

In [ ]:
len(test_pred)